In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

In [ ]:
df1=pd.read_pickle("results_percolation_ALL.pickle")
df2 =pd.read_pickle("results_percolation_002.pickle")

In [ ]:
df = pd.concat([df1,df2], ignore_index = True )
df.head()

In [ ]:
df.phi.unique()
df.sort_values(['ptype','temperature','phi','delta','current_time'])

In [ ]:

grouped1 = df1.sort_values(['temperature','phi','delta']).groupby(['ptype','delta','phi'])
grouped2 = df2.sort_values(['temperature','phi','delta']).groupby(['ptype','delta','phi'])

phi=0.01
fig,(ax1,ax2) = plt.subplots(1,2)
ptype = 'double_manta_asymm_1'
delta = '0.2'
phis=[0.1,0.2,0.3]
for (g1,g2) in zip(grouped1,grouped2):
    
    if g1[0][0] != ptype or g1[0][1] != delta:
        fig,(ax1,ax2) = plt.subplots(1,2)
        ptype = g1[0][0]
        delta = g1[0][1]
   
    plt.title('{},{}'.format(g1[0][0],g1[0][1]))

    if g1[0][2] in phis:
        ax1.plot(g1[1].temperature, g1[1].frac_largest_virtual, label='$\phi =$ {}'.format(g1[0][2]))
        ax1.set_ylim([0,1.1])
        
        ax2.plot(g2[1].temperature, g2[1].frac_largest_virtual, label='$\phi =$ {}'.format(g2[0][2]))
        ax2.set_ylim([0,1.1])
    
    
    plt.legend(loc='best', bbox_to_anchor=(0.8, 0.5, 0.5, 0.5))

plt.show()
    

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import matplotlib as mpl
plt.style.use("seaborn")

df2 = df2.sort_values(['ptype', 'delta','temperature','phi'])

ptypes = df2.ptype.unique()
deltas = df2.delta.unique()

phis = df2.phi.unique()
temps = df2.temperature.unique()



for pi in ptypes:
    for di in deltas:
        fig,ax = plt.subplots(figsize=(10,10))
        arr = df2[(df2['ptype'] == pi) & (df2['delta']== di)]['frac_largest_virtual'].values
        arr = np.reshape(arr, (len(temps),len(phis)))
        
        arr = np.round(arr, decimals=2)
        mpl.rc("legend", fontsize=50)
        heat_map = sns.heatmap(arr,linewidth=1, annot=True, cmap='rainbow')
        plt.title("{},$\delta$ = {}".format(pi,di))
        plt.xlabel("$\phi$")
        plt.ylabel("Temperature")
        
        new_xticks = phis
        plt.xticks(np.arange(0,len(phis)),new_xticks, rotation=45, horizontalalignment='right')
        
        new_yticks = temps
        plt.yticks(np.arange(0,len(temps)),new_yticks)
        heat_map.invert_yaxis()
             
            
                           
                   
        
plt.show()

In [ ]:
np.arange(10)

### Representative networks 
double_manta_asymm_1 
double_mouse_asymm_1
double_mouse_symm_1
double_mouse_symm_2

phi=0.125, delta=0.2, T=0.01

data files:
psi_op.dat
patch_network.dat

Calculate:
average degree
degree distribution 
comunities, 
loops 
draw network with p,np bonds 

In [ ]:
import networkx as nx 
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
def read_bonds(filen):
	first_line_pair = [0,0,0,0]
	cut=False
	with open(filen, 'r') as f:
		network_list = []
		for line in f:
			if "#" in line:
				network_list.append([])
				first_line_pair = [0,0,0,0]
				cut=False

			else:
				line_counter=len(network_list[-1])
				pairs = list(map(int, line.split(" ")))
				if pairs == first_line_pair or cut==True:
					cut=True
				else:
					network_list[-1].append(np.array(pairs))

				if line_counter == 0:
					first_line_pair = pairs
	network_list = [ np.array(item) for item in network_list]

	return network_list


In [ ]:
def read_network(filedir, ptype, phi,delta,T):
    
    dirn="{}/{}/{}_phi_{}_delta_{}_temp_{}".format(filedir,ptype,ptype,phi,delta,T)
    filen="{}/patch_network.dat".format(dirn)
    
    connections = read_bonds(filen)[-1]
 
    G=nx.Graph()
    G.add_edges_from(connections[:,:2])
    
    return G

    
    

In [ ]:
#filedir="/Users/ada/Documents/Code_Development_2020/rhombi/percolations_study/vsc3/percolation_runs/copy_dir"
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

ptype='double_mouse_symm_2'
phi=0.125
delta=0.2
T=0.01 
G=read_network(filedir,ptype,phi,delta,T)



degree_sequence=[ d for n,d in G.degree()]
average_degree = np.mean(degree_sequence)

largest_cc = max(nx.connected_components(G), key=len)
S = G.subgraph(largest_cc).copy()
 


In [ ]:
fig,ax= plt.subplots(figsize=(30,30))

nx.draw(G, pos=nx.spring_layout(G, iterations=500), node_size=70)
plt.show()

In [ ]:
fig,ax= plt.subplots(figsize=(30,30))

nx.draw(G, pos=nx.spring_layout(G, iterations=500), node_size=70)
plt.show()

In [ ]:
fig,ax= plt.subplots(figsize=(30,30))

nx.draw(G, pos=nx.spring_layout(G, iterations=500), node_size=70)
plt.show()

In [ ]:
fig,ax= plt.subplots(figsize=(30,30))

nx.draw(G, pos=nx.spring_layout(G, iterations=500), node_size=70)
plt.show()

In [ ]:
a=G.number_of_edges()
b=G.number_of_nodes()

print(a/b)

In [ ]:
degree_sequence=[ d for n,d in G.degree()]
average_degree = np.mean(degree_sequence)

In [ ]:
average_degree

In [ ]:
N=1500
N_fc = (N*(N-1))/2
print(N_fc)

In [ ]:
a/N_fc

In [ ]:
RG = nx.fast_gnp_random_graph(1500, a/N_fc)
fig,ax= plt.subplots(figsize=(30,30))


largest_cc = max(nx.connected_components(RG), key=len)
RS = RG.subgraph(largest_cc).copy()
 

nx.draw(RS, pos=nx.spring_layout(RS, iterations=500), node_size=70)
plt.show()



In [ ]:
BG = nx.random_degree_sequence_graph(degree_sequence,tries=10)

fig,ax= plt.subplots(figsize=(30,30))
largest_cc = max(nx.connected_components(BG), key=len)
SB = BG.subgraph(largest_cc).copy()
nx.draw(SB, pos=nx.spring_layout(SB, iterations=500), node_size=70)
plt.show()







In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

fig, ax = plt.subplots(figsize=(10,10))
phi=0.125
delta=0.2
T=0.01 

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):
    G=read_network(filedir,ptype,phi,delta,T)
    largest_cc = max(nx.connected_components(G), key=len)
    S = G.subgraph(largest_cc).copy()    
   
    degree_sequence=[ d for n,d in S.degree()]
    average_degree = np.mean(degree_sequence)

    plt.hist(degree_sequence,range=(1,4), density=True,histtype='step',color=color, label=ptype, linewidth=3)
_
plt.xticks(range(1,5))    
plt.legend()
plt.show()



In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

fig, ax = plt.subplots(figsize=(10,10))
phi=0.125
delta=0.3
T=0.01 

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):
    G=read_network(filedir,ptype,phi,delta,T)
    largest_cc = max(nx.connected_components(G), key=len)
    S = G.subgraph(largest_cc).copy()    
   
    degree_sequence=[ d for n,d in S.degree()]
    average_degree = np.mean(degree_sequence)

    plt.hist(degree_sequence,range=(1,4), density=True,histtype='step',color=color, label=ptype, linewidth=3)
_
plt.xticks(range(1,5))    
plt.legend()
plt.show()



In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

fig, ax = plt.subplots(figsize=(10,10))
phi=0.125
delta=0.2
T=0.05 

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):
    G=read_network(filedir,ptype,phi,delta,T)
    largest_cc = max(nx.connected_components(G), key=len)
    S = G.subgraph(largest_cc).copy()    
   
    degree_sequence=[ d for n,d in S.degree()]
    average_degree = np.mean(degree_sequence)

    plt.hist(degree_sequence,range=(1,4), density=True,histtype='step',color=color, label=ptype, linewidth=3)
_
plt.xticks(range(1,5))    
plt.legend()
plt.show()


In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

fig, ax = plt.subplots(figsize=(10,10))
phi=0.1
delta=0.3
T=0.04

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):
    G=read_network(filedir,ptype,phi,delta,T)
    largest_cc = max(nx.connected_components(G), key=len)
    S = G.subgraph(largest_cc).copy()    
   
    degree_sequence=[ d for n,d in S.degree()]
    average_degree = np.mean(degree_sequence)

    plt.hist(degree_sequence,range=(1,4), density=True,histtype='step',color=color, label=ptype, linewidth=3)
_
plt.xticks(range(1,5))    
plt.legend()
plt.show()


In [ ]:
G_test = nx.Graph()
G_test.add_edges_from([[1,2],[1,3],[2,3],[3,7],[2,4],[4,5],[5,6],[6,7],[7,5]])

In [ ]:
print(nx.cycle_basis(G_test))

In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

fig, ax = plt.subplots(figsize=(20,20))
phi=0.1
delta=0.3
T=0.04

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):
    G=read_network(filedir,ptype,phi,delta,T)
    largest_cc = max(nx.connected_components(G), key=len)
    S = G.subgraph(largest_cc).copy()    
   
    cycle_basis=nx.cycle_basis(S)
    cycle_lengths = [ len(c) for c in cycle_basis]

    plt.hist(cycle_lengths,density=True,histtype='step',color=color, label=ptype, linewidth=3)

plt.legend()
plt.show()


In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

fig, ax = plt.subplots(figsize=(20,20))
phi=0.125
delta=0.4
T=0.01

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):
    G=read_network(filedir,ptype,phi,delta,T)
    largest_cc = max(nx.connected_components(G), key=len)
    S = G.subgraph(largest_cc).copy()    
   
    cycle_basis=nx.cycle_basis(S)
    cycle_lengths = [ len(c) for c in cycle_basis]

    plt.hist(cycle_lengths,density=True,bins=100,range=(0,200),histtype='step',color=color, label=ptype, linewidth=3)

    
cycle_basis=nx.cycle_basis(RG)
cycle_lengths = [ len(c) for c in cycle_basis]
plt.hist(cycle_lengths,density=True,bins=100,range=(0,200),histtype='step',color='y', label='ER-random-graph', linewidth=3)


cycle_basis=nx.cycle_basis(BG)
cycle_lengths = [ len(c) for c in cycle_basis]
plt.hist(cycle_lengths,density=True,bins=100,range=(0,200),histtype='step',color='k', label='lim-degree-random-graph', linewidth=3)

    
    
plt.xticks(range(3,20))
plt.legend()
plt.show()


In [ ]:

spectrum_RG=nx.adjacency_spectrum(RG)
plt.hist(spectrum_RG,density=True,bins=100,histtype='step')
plt.show()

In [ ]:
RG1 = nx.fast_gnp_random_graph(100,0.05)
spectrum_RG=nx.adjacency_spectrum(RG1)
plt.plot(spectrum_RG)
plt.show()

In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

fig, ax = plt.subplots(figsize=(20,20))
phi=0.125
delta=0.2
T=0.01

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):
    G=read_network(filedir,ptype,phi,delta,T)
    spectrum_G=nx.adjacency_spectrum(G)
    plt.hist(spectrum_G,density=True,bins=100,histtype='step',color=color, label=ptype, linewidth=3)
    

spectrum_G=nx.adjacency_spectrum(RG)
plt.hist(spectrum_G,density=True,bins=100,histtype='step',color='y', label='ER-random-graph', linewidth=3)

spectrum_G=nx.adjacency_spectrum(BG)
plt.hist(spectrum_G,density=True,bins=100,histtype='step',color='k', label='lim-degree-random-graph', linewidth=3)


plt.legend()
plt.show()


In [ ]:
comm_RG=nx.algorithms.community.label_propagation.asyn_lpa_communities(RG)


In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"

fig, ax = plt.subplots(figsize=(20,20))
phi=0.125
delta=0.2
T=0.01

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):
    G=read_network(filedir,ptype,phi,delta,T)
    comm_G=nx.algorithms.community.label_propagation.asyn_lpa_communities(G)
    len_comm = [ len(c) for c in comm_G]

    plt.hist(len_comm,density=True,bins=10,histtype='step',color=color, label=ptype, linewidth=3)

comm_G=nx.algorithms.community.label_propagation.asyn_lpa_communities(RG)
len_comm = [ len(c) for c in comm_G]
plt.hist(len_comm,density=True,bins=10,histtype='step',color='y', label='ER-random-graph', linewidth=3)

comm_G=nx.algorithms.community.label_propagation.asyn_lpa_communities(BG)
len_comm = [ len(c) for c in comm_G]
plt.hist(len_comm,density=True,bins=10,histtype='step',color='k', label='lim-degree-random-graph', linewidth=3)


plt.legend()
plt.show()

In [ ]:
print(len(list(nx.bridges(RG))), len(list(nx.bridges(G))))

In [ ]:
nx.has_bridges(G)

In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"
phi=0.15

delta=0.3
T=0.01

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype, color in zip(ptypes, colors):

    G=read_network(filedir,ptype,phi,delta,T)
    
    largest_cc = max(nx.connected_components(G), key=len)
    S = G.subgraph(largest_cc).copy()    

    print(ptype)
    
    # average shortest path 
    print("average shortest paths", nx.average_shortest_path_length(S))
    
    # betweenness centraliaty 
    bc = nx.betweenness_centrality(S)
    values=np.zeros(len(G.nodes()))
    
    for key in bc:
        values[key-1] = bc[key] 
        
    print(np.min(values), np.max(values))
    fig,ax=plt.subplots(figsize=(30,30))
    plt.title(ptype)
    nx.draw(G,pos=nx.spring_layout(G, iterations=500), node_size=70, cmap=plt.get_cmap('viridis'), node_color=values)
    
# Random graph 

bc = nx.betweenness_centrality(RS)
values=np.zeros(len(G.nodes()))
for key in bc:
    values[key-1] = bc[key] 
    
print(np.min(values), np.max(values))
fig,ax=plt.subplots(figsize=(30,30))
plt.title("E-R-random-graph")
nx.draw(RG,pos=nx.spring_layout(RG, iterations=500), node_size=70, cmap=plt.get_cmap('viridis'), node_color=values)
print(nx.average_shortest_path_length(RS))

print(np.min(values), np.max(values))
bc = nx.betweenness_centrality(SB)
values=np.zeros(len(G.nodes()))
for key in bc:
    values[key-1] = bc[key] 

fig,ax=plt.subplots(figsize=(30,30))
plt.title("same-sequence-graph")
nx.draw(BG,pos=nx.spring_layout(BG, iterations=500), node_size=70, cmap=plt.get_cmap('viridis'), node_color=values)
print(nx.average_shortest_path_length(SB))

plt.show()


In [ ]:
plt.show()

In [ ]:
filedir="/home/carina/Documents/2D_patchy/percolation_study/vsc3/runs/runs"
phi=0.15

delta=0.3
T=0.01

colors=['r','g','b','m']
ptypes= ['double_manta_asymm_1', 'double_mouse_asymm_1', 'double_mouse_symm_1', 'double_mouse_symm_2']

for ptype1 in ptypes:
    for ptype2 in ptypes:
        if ptype1 != ptype2:
            G1=read_network(filedir,ptype1,phi,delta,T)
            G2=read_network(filedir,ptype2,phi,delta,T)
            print(ptype1,ptype2, nx.graph_edit_distance(G1, G2))